In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

In [2]:
#from source.utils import *
from source.preprocess import *
import pickle
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm

In [3]:
path = "../../Desktop/DLC_social_1/"

# Set up and design the project

In [4]:
with open('{}DLC_social_1_exp_conditions.pickle'.format(path), 'rb') as handle:
    Treatment_dict = pickle.load(handle)

In [5]:
#Which angles to compute?
bp_dict = {'B_Nose':['B_Left_ear','B_Right_ear'],
          'B_Left_ear':['B_Nose','B_Right_ear','B_Center','B_Left_flank'],
          'B_Right_ear':['B_Nose','B_Left_ear','B_Center','B_Right_flank'],
          'B_Center':['B_Left_ear','B_Right_ear','B_Left_flank','B_Right_flank','B_Tail_base'],
          'B_Left_flank':['B_Left_ear','B_Center','B_Tail_base'],
          'B_Right_flank':['B_Right_ear','B_Center','B_Tail_base'],
          'B_Tail_base':['B_Center','B_Left_flank','B_Right_flank']}

In [6]:
%%time
DLC_social_1 = project(path=path,#Path where to find the required files
                   smooth_alpha=0.85,                    #Alpha value for exponentially weighted smoothing
                   distances=['B_Center','B_Nose','B_Left_ear','B_Right_ear','B_Left_flank',
                              'B_Right_flank','B_Tail_base'],
                   ego=False,
                   angles=True,
                   connectivity=bp_dict,
                   arena='circular',                  #Type of arena used in the experiments
                   arena_dims=[380],                  #Dimensions of the arena. Just one if it's circular
                   video_format='.mp4',
                   table_format='.h5',
                   exp_conditions=Treatment_dict)

CPU times: user 2.68 s, sys: 794 ms, total: 3.48 s
Wall time: 1.15 s


# Run project

In [7]:
%%time
DLC_social_1_coords = DLC_social_1.run(verbose=True)
print(DLC_social_1_coords)
type(DLC_social_1_coords)

Loading trajectories...
Smoothing trajectories...
Computing distances...
Computing angles...
Done!
Coordinates of 47 videos across 4 conditions
CPU times: user 10.3 s, sys: 612 ms, total: 10.9 s
Wall time: 11.2 s


source.preprocess.coordinates

# Generate coords

In [8]:
%%time
ptest = DLC_social_1_coords.get_coords(center=True, polar=False, speed=0, length='00:10:00')
ptest._type

CPU times: user 912 ms, sys: 59.8 ms, total: 972 ms
Wall time: 911 ms


'coords'

In [9]:
%%time
dtest = DLC_social_1_coords.get_distances(speed=0, length='00:10:00')
dtest._type

CPU times: user 536 ms, sys: 373 ms, total: 909 ms
Wall time: 909 ms


'dists'

In [10]:
%%time
atest = DLC_social_1_coords.get_angles(degrees=True, speed=0, length='00:10:00')
atest._type

CPU times: user 129 ms, sys: 76.7 ms, total: 205 ms
Wall time: 205 ms


'angles'

# Visualization playground

In [11]:
#ptest.plot_heatmaps(['B_Center', 'W_Center'], i=1)

In [12]:
#Plot animation of trajectory over time with different smoothings
#plt.plot(ptest['Day2Test13DLC']['B_Center'].iloc[:5000]['x'],
#         ptest['Day2Test13DLC']['B_Center'].iloc[:5000]['y'], label='alpha=0.85')

#plt.xlabel('x')
#plt.ylabel('y')
#plt.title('Mouse Center Trajectory using different exponential smoothings')
#plt.legend()
#plt.show()

# Dimensionality reduction playground

In [13]:
#pca = ptest.pca(4, 1000)

In [14]:
#plt.scatter(*pca[0].T)
#plt.show()

# Preprocessing playground

In [43]:
mtest = merge_tables(DLC_social_1_coords.get_coords(center=True, polar=True, length='00:10:00'))#,
#                      DLC_social_1_coords.get_distances(speed=0, length='00:10:00'),
#                      DLC_social_1_coords.get_angles(degrees=True, speed=0, length='00:10:00'))

In [44]:
#pptest = mtest.preprocess(window_size=51, filter='gaussian', sigma=10, shift=20)

In [45]:
pttest = mtest.preprocess(window_size=11, window_step=6, filter=None, standard_scaler=True)
pttest.shape

(117507, 11, 28)

In [46]:
#plt.plot(pttest[2,:,2], label='normal')
#plt.plot(pptest[2,:,2], label='gaussian')
#plt.legend()
#plt.show()

# Trained models playground

### Seq 2 seq Variational Auto Encoder

In [70]:
from datetime import datetime
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.constraints import UnitNorm
from tensorflow.keras.layers import Bidirectional, Dense, Dropout
from tensorflow.keras.layers import Lambda, LSTM
from tensorflow.keras.layers import RepeatVector, TimeDistributed
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from source.model_utils import *
import keras as k
import tensorflow as tf

In [71]:
NAME = 'Baseline_VAE_short_512'
log_dir = os.path.abspath(
    "logs/fit/{}_{}".format(NAME, datetime.now().strftime("%Y%m%d-%H%M%S"))
)
tensorboard_callback = k.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [72]:
from source.models import SEQ_2_SEQ_AE, SEQ_2_SEQ_VAE, SEQ_2_SEQ_VAEP

In [73]:
encoder, decoder, ae = SEQ_2_SEQ_AE(pttest.shape).build()

In [93]:
encoder, generator, vae, kl_wu, mmd_wu = SEQ_2_SEQ_VAE(pttest.shape,
                                         loss='ELBO+MMD',
                                         kl_warmup_epochs=5,
                                         mmd_warmup_epochs=5).build()

In [75]:
encoder, generator, vaep = SEQ_2_SEQ_VAEP(pttest.shape).build()

In [76]:
#ae.summary()
#vae.summary()
#vaep.summary()

In [77]:
#from tensorflow.keras.utils import plot_model
#plot_model(vaep, show_shapes=True)

In [55]:
#plot_model(vae)

In [56]:
#np.random.shuffle(pttest)
pttrain = pttest[:-15000]
pttest  = pttest[-15000:]
pttrain = pttrain[:15000]

In [57]:
#lr_schedule = tf.keras.callbacks.LearningRateScheduler(
#    lambda epoch: 1e-3 * 10**(epoch / 20))

In [94]:
#tf.config.experimental_run_functions_eagerly(False)
history = vae.fit(x=pttrain[:-1], y=pttrain[:-1], epochs=100, batch_size=512, verbose=1,
                  validation_data=(pttest[:-1], pttest[:-1]),
                  callbacks=[tensorboard_callback, kl_wu, mmd_wu])

Train on 14999 samples, validate on 14999 samples
Epoch 1/100
14848/14999 [============================>.] - ETA: 0s - loss: 1442235.6078 - mae: 0.7399beta: tf.Tensor(0.0, shape=(), dtype=float32)
mmd_beta: tf.Tensor(0.0, shape=(), dtype=float32)
14999/14999 [==============================] - 49s 3ms/sample - loss: 1429936.8417 - mae: 0.7378 - val_loss: 1590634.8626 - val_mae: 0.8145
Epoch 2/100
14848/14999 [============================>.] - ETA: 0s - loss: 615069.0938 - mae: 0.4825beta: tf.Tensor(0.2, shape=(), dtype=float32)
mmd_beta: tf.Tensor(0.2, shape=(), dtype=float32)
14999/14999 [==============================] - 46s 3ms/sample - loss: 610375.3203 - mae: 0.4820 - val_loss: 1520268.2814 - val_mae: 0.8007
Epoch 3/100
14848/14999 [============================>.] - ETA: 0s - loss: 459210.1994 - mae: 0.4111beta: tf.Tensor(0.4, shape=(), dtype=float32)
mmd_beta: tf.Tensor(0.4, shape=(), dtype=float32)
14999/14999 [==============================] - 50s 3ms/sample - loss: 455880.9639 

KeyboardInterrupt: 

In [ ]:
#tf.config.experimental_run_functions_eagerly(False)
#history = vaep.fit(x=pttrain[:-1], y=[pttrain[:-1],pttrain[1:]], epochs=100, batch_size=512, verbose=1,
#                   validation_data=(pttest[:-1], [pttest[:-1],pttest[1:]]),
#                   callbacks=[tensorboard_callback])